In [16]:
import re
import requests
import json
from bs4 import BeautifulSoup
import crawler.crawl as crawl
from datetime import datetime, timedelta
import dateparser
import importlib
importlib.reload(crawl)


class CrawlCafeF:
    base_url = 'https://cafef.vn'
    page = 1
    daily = False # if exist in database will kill process.

    def __init__(self):
        pass

    def get_response(self, url, params = {}):
        headers = {
            'Access-Control-Allow-Origin': '*',
            'Access-Control-Allow-Credentials': 'true',
            'Access-Control-Allow-Methods': 'GET, HEAD, POST, PUT, DELETE, TRACE, OPTIONS, PATCH',
            'Access-Control-Allow-Headers': 'X-Real-IP,X-AGENT,Pragma,X-REFERER,X-AUTH-TOKEN,Accept-Encoding,channel,X-XSS-Protection,X-Content-Type-Options,Strict-Transport-Security,Content-Type,Authorization,Accept,Origin,User-Agent,DNT,Cache-Control,X-Mx-ReqToken,Keep-Alive,X-Requested-With,If-Modified-Since,token-id',
            'Content-Encoding': 'gzip'
        }
        # Make an HTTP GET request to the API endpoint using the requests library
        response = requests.get(url, params=params, headers=headers)
        return response
    
    def get_page_detail(self, url, params = {}):
        url = self.base_url + url
        print('get_page_detail', url, params)
        response = self.get_response(url, params)
        if response.status_code // 100 == 2:
            content_html = response.content
            soup = BeautifulSoup(content_html, 'html.parser')
            # Find the title element
            publishdate_elem = soup.find('span', attrs={'data-role': 'publishdate'})
            content_detail_elem = soup.find('div', attrs={'class': 'contentdetail'})
        
            if publishdate_elem:
                publishDate = publishdate_elem.get_text()
                publishDate = publishDate.strip()
                publishDate = dateparser.parse(publishDate)
            else:
                publishDate = None

            content_detail = content_detail_elem.get_text() if content_detail_elem else None

            return {
                'date': publishDate,
                'content': content_detail
            }
        else:
            # If unsuccessful, print the status code and reason for failure
            print(f"Request failed with status code {response.status_code}: {response.reason}")
            return {}

    def crawl_html(self, url, params = {}):
        print('crawl_html', url, params)
        response = self.get_response(url, params)
        if response.status_code // 100 == 2:
            content_html = response.content

            # Parse the HTML using BeautifulSoup
            soup = BeautifulSoup(content_html, 'html.parser')

            # Find the title element
            title_elements = soup.select('.box-category-item h3 a')
            if title_elements is None or title_elements is []:
                return False

            for title_element in title_elements:
                title = title_element.get_text()
                link = title_element['href']

                print("Title:", title)
                print("Link:", link)
                data_detail = self.get_page_detail(link, {})

                item = {
                    'domain': 'https://cafef.vn/timelinelist/18831',
                    'title': title,
                    'url': link,
                    'date': data_detail['date'] if 'date' in data_detail else None,
                    'content': data_detail['content'] if 'content' in data_detail else None,
                }
                isDup = crawl.create_article(item, not self.daily)
                return isDup 
        else:
            # If unsuccessful, print the status code and reason for failure
            print(f"Request failed with status code {response.status_code}: {response.reason}")
            return False

    def run(self, page = {"from": None, "to": None}, daily = False):
        self.daily = daily
        page_from = 1 if page['from'] is None else page['from']
        page_to = -1 if page['to'] is None else page['to']
        while (True):
            isSuccess = self.crawl_html('https://cafef.vn/timelinelist/18831/' + str(page_from) + '.chn', {})
            if isSuccess == False or page_to == page_from:
                break
            page_from += 1


CrawlCafeF().run(daily=False, page={"from": 10, "to": -1})
# obj.get_page_detail('/von-ngoai-bat-dau-tang-toc-185230818163354771.htm')
# crawl_json('https://finfo-api.vndirect.com.vn/v4/news?q=newsType:company_report~locale:VN~newsSource:VNDIRECT&sort=newsDate:desc~newsTime:desc&size=20&page=3', {})

crawl_html https://cafef.vn/timelinelist/18831/10.chn {}
Title: Một doanh nghiệp bất động sản lên kế hoạch lợi nhuận năm 2023 cao gấp 18 lần
Link: /mot-doanh-nghiep-bat-dong-san-len-ke-hoach-loi-nhuan-nam-2023-cao-gap-18-lan-188230821111223728.chn
get_page_detail https://cafef.vn/mot-doanh-nghiep-bat-dong-san-len-ke-hoach-loi-nhuan-nam-2023-cao-gap-18-lan-188230821111223728.chn {}
crawl_html https://cafef.vn/timelinelist/18831/11.chn {}
Title: Quỹ tỷ USD của Dragon Capital nâng lượng tiền nắm giữ lên cao nhất 9 tháng ngay trước tuần VN-Index giảm mạnh
Link: /quy-ty-usd-cua-dragon-capital-nang-luong-tien-nam-giu-len-cao-nhat-9-thang-ngay-truoc-tuan-vn-index-giam-manh-188230819222906828.chn
get_page_detail https://cafef.vn/quy-ty-usd-cua-dragon-capital-nang-luong-tien-nam-giu-len-cao-nhat-9-thang-ngay-truoc-tuan-vn-index-giam-manh-188230819222906828.chn {}
crawl_html https://cafef.vn/timelinelist/18831/12.chn {}
Title: Bidiphar (DBD) được chấp thuận đầu tư dự án nhà máy sản xuất thuốc vô

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))